In [1]:
import os
import numpy as np
import xgboost as xgb

In [2]:
dtrain = xgb.DMatrix('agaricus.txt.train')
dtest = xgb.DMatrix('agaricus.txt.test')

[15:34:27] 6513x127 matrix with 143286 entries loaded from agaricus.txt.train
[15:34:27] 1611x127 matrix with 35442 entries loaded from agaricus.txt.test


In [3]:
setattr(dtrain, 'xyz', 1)

In [4]:
param = {'max_depth': 2, 'eta': 1}
watchlist = [(dtest, 'eval'), (dtrain, 'train')]
num_round = 2

In [5]:

# user define objective function, given prediction, return gradient and second
# order gradient this is log likelihood loss
def logregobj(preds, dtrain):
    print(dtrain.xyz)
    labels = dtrain.get_label()
    preds = 1.0 / (1.0 + np.exp(-preds))
    grad = preds - labels
    hess = preds * (1.0 - preds)
    print(grad.shape, hess.shape)
    return grad, hess


In [6]:
# NOTE: when you do customized loss function, the default prediction value is
# margin. this may make builtin evaluation metric not function properly for
# example, we are doing logistic loss, the prediction is score before logistic
# transformation the builtin evaluation error assumes input is after logistic
# transformation Take this in mind when you use the customization, and maybe
# you need write customized evaluation function
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    # return a pair metric_name, result. The metric name must not contain a
    # colon (:) or a space since preds are margin(before logistic
    # transformation, cutoff at 0)
    return 'my-error', float(sum(labels != (preds > 0.0))) / len(labels)

In [7]:
# training with customized objective, we can also do step by step training
# simply look at xgboost.py's implementation of train
bst = xgb.train(param, dtrain, num_round, watchlist, obj=logregobj,
                feval=evalerror)

1
(6513,) (6513,)
[0]	eval-rmse:1.59229	train-rmse:1.59596	eval-my-error:0.04283	train-my-error:0.04652
1
(6513,) (6513,)
[1]	eval-rmse:2.40520	train-rmse:2.40977	eval-my-error:0.02173	train-my-error:0.02226
